In [1]:
import rasterio
from rio_tiler.io import Reader
from vsifile.rasterio import opener

import logging

logging.basicConfig(level=logging.DEBUG)

# turn off some logs
logging.getLogger("rasterio").setLevel(logging.ERROR)
logging.getLogger("httpcore").setLevel(logging.ERROR)
logging.getLogger("botocore").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)


src_path = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/T/VK/2023/10/S2B_15TVK_20231008_0_L2A/TCI.tif"
z, x, y = 11, 493, 741


In [2]:
import attr
from rio_tiler.io import Reader

@attr.s
class VSIReader(Reader):

    def __attrs_post_init__(self):
        if not self.dataset:
            self.dataset = self._ctx_stack.enter_context(rasterio.open(self.input, opener=opener))
        super().__attrs_post_init__()

In [4]:
%%timeit -n 1 -r 1

with rasterio.Env(
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    GDAL_INGESTED_BYTES_AT_OPEN=32768,
    GDAL_HTTP_MERGE_CONSECUTIVE_RANGES="YES",
    CPL_DEBUG="ON",
    CPL_CURL_VERBOSE="YES",
    GDAL_HTTP_MULTIRANGE="YES",
    GDAL_HTTP_VERSION="2",
):
    with VSIReader(src_path) as src:
        src.preview()

DEBUG:vsifile:Opening: test (mode: rb)


DEBUG:vsifile:Opening: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/T/VK/2023/10/S2B_15TVK_20231008_0_L2A/TCI.tif (mode: rb)
DEBUG:vsifile:Found Header in cache
DEBUG:vsifile:Reading 0->32768 from Header cache
DEBUG:vsifile:Reading 0->8 from Header cache
DEBUG:vsifile:Reading 8->10 from Header cache
DEBUG:vsifile:Reading 10->238 from Header cache
DEBUG:vsifile:Reading 238->242 from Header cache
DEBUG:vsifile:Reading 242->248 from Header cache
DEBUG:vsifile:Reading 1216->1222 from Header cache
DEBUG:vsifile:Reading 1304->1328 from Header cache
DEBUG:vsifile:Reading 1328->1376 from Header cache
DEBUG:vsifile:Reading 1376->1440 from Header cache
DEBUG:vsifile:Reading 1440->1470 from Header cache
DEBUG:vsifile:Reading 1222->1303 from Header cache
DEBUG:vsifile:Reading 1470->1472 from Header cache
DEBUG:vsifile:Reading 1472->1652 from Header cache
DEBUG:vsifile:Reading 1652->1656 from Header cache
DEBUG:vsifile:Reading 1656->1662 from Header cache
DEBUG:vsifile:R

2.49 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n 1 -r 1

with rasterio.Env(
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    GDAL_INGESTED_BYTES_AT_OPEN=32768,
    GDAL_HTTP_MERGE_CONSECUTIVE_RANGES="YES",
    CPL_DEBUG="ON",
    CPL_CURL_VERBOSE="YES",
    GDAL_HTTP_MULTIRANGE="YES",
    GDAL_HTTP_VERSION="2",
):
    with VSIReader(src_path) as src:
        src.tile(x, y, z)

DEBUG:vsifile:Opening: test (mode: rb)
DEBUG:vsifile:Opening: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/T/VK/2023/10/S2B_15TVK_20231008_0_L2A/TCI.tif (mode: rb)
DEBUG:vsifile:Found Header in cache
DEBUG:vsifile:Reading 0->32768 from Header cache
DEBUG:vsifile:Reading 0->8 from Header cache
DEBUG:vsifile:Reading 8->10 from Header cache
DEBUG:vsifile:Reading 10->238 from Header cache
DEBUG:vsifile:Reading 238->242 from Header cache
DEBUG:vsifile:Reading 242->248 from Header cache
DEBUG:vsifile:Reading 1216->1222 from Header cache
DEBUG:vsifile:Reading 1304->1328 from Header cache
DEBUG:vsifile:Reading 1328->1376 from Header cache
DEBUG:vsifile:Reading 1376->1440 from Header cache
DEBUG:vsifile:Reading 1440->1470 from Header cache
DEBUG:vsifile:Reading 1222->1303 from Header cache
DEBUG:vsifile:Reading 1470->1472 from Header cache
DEBUG:vsifile:Reading 1472->1652 from Header cache
DEBUG:vsifile:Reading 1652->1656 from Header cache
DEBUG:vsifile:Reading 1656-

9.44 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
